In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
print "Tensorflow Version:", tf.__version__

In [85]:
zip_inc = pd.read_excel('zip_to_income.xlsx', header=None)

zipList = zip_inc[0].get_values().tolist()
incList = zip_inc[1].get_values().tolist()

zip_to_income = dict()
for i,j in zip(zipList, incList):
    zip_to_income[i] = j

avgHouseIncome = 59039

In [92]:
data = pd.read_excel('resd_data_1.xlsx', header=None).values
n_train = 3 * data.shape[0] // 4
cols = data.shape[1]

In [93]:
for count, zipCode in enumerate(data[:, cols - 2]):
    if zipCode in zip_to_income.keys():
        data[count, cols - 2] = zip_to_income[zipCode]
    else:
        data[count, cols - 2] = avgHouseIncome

In [94]:
(train_data, train_targets) = data[0:n_train, 0:cols - 1], data[0:n_train, cols - 1]
(test_data,  test_targets)  = data[n_train::, 0:cols - 1], data[n_train::, cols - 1]

In [95]:
train_data.shape

(7500, 7)

In [96]:
test_data.shape

(2500, 7)

In [103]:
# Normalizing the data
mean = train_data.mean(axis=0)
train_data = train_data - mean
test_data  = test_data - mean

std = train_data.std(axis=0)
train_data /= std
test_data /= std

In [104]:
# Building the model network
def build_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [ ]:
# K-fold Validation
k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []
for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)
    
    model = build_model()
    model.fit(partial_train_data, partial_train_targets,
              epochs=num_epochs, batch_size=1, verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

('processing fold #', 0)
('processing fold #', 1)
